# Mood Mirror

Una aplicación en Python que permite al usuario ingresar un texto libre, como una entrada de diario, y ofrece:

- La emoción principal (por ejemplo, alegría, tristeza, enojo, miedo, sorpresa).
- Un mensaje personalizado según la emoción detectada.
- Un gráfico que muestra cómo varían tus emociones a lo largo del tiempo.


---

### Importaciones

In [2]:
import pandas as pd

### Contamos las filas
Hacemos esto para saber los datos y para poder cumplir con la consigna

In [ ]:
for split in ["train", "val", "test"]:
    path = f"{split}.txt"
    df_split = pd.read_csv(
        path,
        sep="\t",
        header=None,
        names=["text", "emotion"]
    )
    print(f"--- {split.upper()} ---")
    print("Total ejemplos:", len(df_split))
    print(df_split["emotion"].value_counts(), "\n")

--- TRAIN ---
Total ejemplos: 16000
Series([], Name: count, dtype: int64) 

--- VAL ---
Total ejemplos: 2000
Series([], Name: count, dtype: int64) 

--- TEST ---
Total ejemplos: 2000
Series([], Name: count, dtype: int64) 

